In [109]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")


epsilon = 1.0e-8

def equal(a, b):
    return abs(a - b) < epsilon

class Tableau:
    def __init__(self, A, b, c):
        self.mat = np.hstack([np.vstack([np.zeros(1), b]),
                              np.vstack([c,A])])
        self.m = self.mat.shape[0]
        self.n = self.mat.shape[1]
        self.counter = 0


def nl(k):
    print('-' * k)

def print_tableau(tab, mes):
    tab.counter += 1
    print(f"\n{tab.counter}. Базис {mes}:\n")
    nl(70)
    columns = ['col:','b[i]'] + [f'x{j}' for j in range(1,tab.n)]
    data = []
    for i in range(tab.m):
        cur_row = []
        if i == 0:
            cur_row.append('max:')
        else:
            cur_row.append(f'b{i}:')
        for j in range(tab.n):
            cur_row.append(f'{tab.mat[i][j]:8.2f}')
        data.append(cur_row)
    print(pd.DataFrame(data = data, columns = columns))
    nl(70)

def pivot_on(tab, row, col):
    pivot = tab.mat[row][col]
    assert pivot > 0
    tab.mat[row] /= pivot
    assert equal(tab.mat[row][col], 1.)

    for i in range(tab.m):
        if i == row:
            continue
        multiplier = tab.mat[i][col]
        tab.mat[i] -= multiplier * tab.mat[row]

def find_pivot_column(tab):
    pivot_col = 1
    lowest = tab.mat[0][pivot_col]
    for j in range(1, tab.n):
        if tab.mat[0][j] < lowest:
            lowest = tab.mat[0][j]
            pivot_col = j
    print(f"Минимальное отрицательное значение в row[0] это col[{pivot_col}] = {lowest}.\n")
    if lowest >= 0:
        return -1
    return pivot_col

def find_pivot_row(tab, pivot_col):
    pivot_row = 0
    min_ratio = -1
    print(f"Частные по ограничениям A[row_i,0]/A[row_i,{pivot_col}] = [", end='')
    for i in range(1, tab.m):
        ratio = tab.mat[i][0] / tab.mat[i][pivot_col]
        print(f"{ratio:8f}, ", end='')
        if ((ratio > 0 and ratio < min_ratio) or min_ratio < 0) and tab.mat[i][pivot_col]>0:
            min_ratio = ratio
            pivot_row = i
    print("].\n")
    if min_ratio == -1:
        return -1
    print(f"Найдено целевое значение A[{pivot_row},{pivot_col}], минимальное положительное={min_ratio} в ряду={pivot_row}.\n")
    return pivot_row

def add_slack_variables(tab):
    slack_vars = np.eye(tab.m - 1)
    new_mat = np.zeros((tab.m, tab.n + tab.m - 1))
    new_mat[:, :tab.n] = tab.mat
    new_mat[1:, tab.n:] = slack_vars
    tab.mat = new_mat
    tab.n = tab.n + tab.m - 1

def check_b_positive(tab):
    assert all(tab.mat[i][0] >= 0 for i in range(1, tab.m))

def find_basis_variable(tab, col):
    xi = -1
    for i in range(1, tab.m):
        if equal(tab.mat[i][col], 1):
            if xi == -1:
                xi = i
            else:
                return -1
        elif not equal(tab.mat[i][col], 0):
            return -1
    return xi

def print_optimal_vector(tab, message):
    print(f"{message}: ", end='')
    for j in range(1, tab.n):
        xi = find_basis_variable(tab, j)
        if xi != -1:
            print(f"x{j}={tab.mat[xi][0]:8f}, ", end='')
        else:
            print(f"x{j}=0, ", end='')
    print('\n')

def simplex(tab):
    add_slack_variables(tab)
    check_b_positive(tab)
    print_tableau(tab, "Базис с добавленными дополнительными переменными")

    loop = 0
    while True:
        loop += 1
        pivot_col = find_pivot_column(tab)
        if pivot_col < 0:
            print(f"Найдено оптимальное значение=A[0,0]={tab.mat[0][0]:8f} (нет негативных в ряду 0).\n")
            print_optimal_vector(tab, "Оптимальный вектор")
            break
        print(f"Переменная x{pivot_col} будет базовой, целевая колонка={pivot_col}.\n")

        pivot_row = find_pivot_row(tab, pivot_col)
        if pivot_row < 0:
            print("Неограничена (нет целевого ряда).\n")
            break
        print(f"Выбираем переменную x{pivot_row}, Целевая колонка={pivot_row}\n")

        pivot_on(tab, pivot_row, pivot_col)
        print_tableau(tab, "После преобразования")
        print_optimal_vector(tab, "Возможное базовое решение")

        if loop > 50:
            print(f"Слишком много итераций > {loop}.\n")
            break

In [110]:
def is_float(element: any) -> bool:
    if element is None: 
        return False
    try:
        float(element)
        return True
    except ValueError:
        return False
    
def read_from_file(inputfile):
    file =  open(inputfile)
    data = file.read().splitlines()
    allowed_signs = {'<=','>='}
    if data[0] == 'min':
        min_max_multiplicator = 1
    elif data[0] == 'max':
        min_max_multiplicator = -1
    else:
        raise AssertionError('Введите направление оптимизации в первой строке файла')
    c = list(map(float, data[1].split()))
    A = []
    b = []
    for row in data[2:]:
        cur_row = []
        cur_sign = 0 
        for val in row.split():
            if is_float(val):
                cur_row.append(float(val))
            elif val in allowed_signs and cur_sign == 0:
                if val == '<=':
                    cur_sign = 1
                else:
                    cur_sign = -1
            else:
                raise AssertionError(f'Неправильный формат строки {row}')
        if cur_sign != 0:
            cur_row = np.array(cur_row) * cur_sign
        else:
            raise AssertionError(f'Неправильный формат строки, знак неравенства не найден {row} \n Допустимые форматы знаков неравенств: {allowed_signs}')
        A.append(cur_row[:-1])
        b.append(cur_row[-1])
    A = np.array(A)
    b = np.array(b).reshape(-1,1)
    c = np.array(c).reshape(1, -1) * min_max_multiplicator
    
    return A, b, c    

In [111]:
inputfile = 'test.txt'

Переменные решения:
$ x_1 $ - количество шкафов  
$ x_2 $ - количество тумб 

Целевая функция:
$$ 200*x_1 + 100*x_2 $$

Ограничения:

$$ 
\begin{cases} 
3.5*x_1 + x_2 <= 350\\ 
x_1 + 2*x_2 <= 240\\
x_1 + x_2 <= 150\\
x_1 >= 0\\
x_2 >= 0\\
\end{cases}$$

In [112]:
A,b,c = read_from_file(inputfile)
simplex(Tableau(A, b, c))


1. Базис Базис с добавленными дополнительными переменными:

----------------------------------------------------------------------
   col:      b[i]        x1        x2        x3        x4        x5        x6  \
0  max:      0.00   -200.00   -100.00      0.00      0.00      0.00      0.00   
1   b1:    350.00      3.50      1.00      1.00      0.00      0.00      0.00   
2   b2:    240.00      1.00      2.00      0.00      1.00      0.00      0.00   
3   b3:    150.00      1.00      1.00      0.00      0.00      1.00      0.00   
4   b4:     -0.00     -1.00     -0.00      0.00      0.00      0.00      1.00   
5   b5:     -0.00     -0.00     -1.00      0.00      0.00      0.00      0.00   

         x7  
0      0.00  
1      0.00  
2      0.00  
3      0.00  
4      0.00  
5      1.00  
----------------------------------------------------------------------
Минимальное отрицательное значение в row[0] это col[1] = -200.0.

Переменная x1 будет базовой, целевая колонка=1.

Частные по огран

# Попробуем использовать языковую модель для формирования математической постановки задачи

In [22]:
from gigachat import GigaChat


giga = GigaChat(credentials=..., scope="GIGACHAT_API_PERS", verify_ssl_certs=False)

In [26]:
text = "Запиши в математической постановке задачи линейного программирования: \
Цех может выпускать два вида продукции: шкафы и тумбы для\
телевизора. На каждый шкаф расходуется 3,5 кв. м стандартных ДСП, 1 кв. м\
листового стекла и 1 человеко-день трудозатрат. На тумбу – 1кв. м ДСП,\
2 кв. м стекла и 1 человеко-день трудозатрат. Прибыль от продажи 1 шкафа\
составляет 200 у.е., а 1 тумбы – 100 у.е.\
Материальные и трудовые ресурсы ограничены: в цехе работают\
150 рабочих, в день нельзя израсходовать больше 350 кв. м ДСП и более\
240 кв. м стекла.\
Какое количество шкафов и тумб должен выпускать цех, чтобы\
сделать прибыль максимальной?"

In [27]:
response = giga.chat(text)

К сожлению, на запросы к API модель выдает достаточно слабые ответы. В сравнении с ответами из веб-интерфейса Чат-бота. Подробнее в отчете.

In [38]:
print(response.choices[0].message.content)

Минимальная прибыль будет при выпуске только тумб, так как на тумбу расходуется меньше ресурсов, чем на шкаф. Максимальная прибыль будет при выпуске только шкафов, так как на шкаф расходуется меньше ресурсов, чем на тумбу. 
Минимум по ресурсам будет при выпуске только шкафов, так как на шкаф расходуется меньше ресурсов, чем на тумбу. 
Максимум по времени будет при выпуске только тумб, так как на тумбу расходуется меньше ресурсов, чем на шкаф. 
Таким образом, оптимальным решением будет выпуск только шкафов.


Попытка использовать prompt, интерфейс которых предусмотрен в API, для формирования ответов не увенчалась успехом.

In [20]:
from langchain.schema import HumanMessage, SystemMessage
from langchain.chat_models.gigachat import GigaChat

# Авторизация в сервисе GigaChat
chat = GigaChat(credentials=..., verify_ssl_certs=False)

messages = [
    SystemMessage(
        content="Ты бот-математик, который текстовые условия задачи записывает в виде математических формулировок задач линейного программирования."
    )
]

while(True):
    user_input = input("User: ")
    messages.append(HumanMessage(content=user_input))
    res = chat(messages)
    messages.append(res)
    print("Bot: ", res.content)
    break

Bot:  Максимальная прибыль будет тогда, когда выпускается товар с наибольшей стоимостью. Для шкафов это 200 у.е., для тумб — 100 у.е. Таким образом, цех должен выпускать тумбы.
